In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv
from uuid import uuid4
import os

from api.agents.prompts.sql_agent import system_prompt

load_dotenv()

True

In [32]:
def create_sql_agent(model: str, db_uri: str, debug: bool, as_tool=False):
	@tool
	def sql_agent(question: str):
		"""Use the database to answer the input question.

		Args:
			question: question to the database answer.
		"""
		interactions = agent.invoke({"messages": [{"role": "user", "content": question}]})
		return interactions["messages"][-1].content

	llm = init_chat_model(model)
	db = SQLDatabase.from_uri(db_uri)

	toolkit = SQLDatabaseToolkit(db=db, llm=llm)
	tools = toolkit.get_tools()

	agent = create_agent(
		model=llm,
		tools=tools,
		system_prompt=system_prompt.format(
			dialect=db.dialect,
			top_k=5,
		),
		debug=debug
	)

	return sql_agent if as_tool else agent


In [59]:
def create_conversational_agent(model: str, tools: list, checkpointer=None, debug: bool = False):
    llm = init_chat_model(model)
    agent = create_agent(
        model=llm,
        tools=tools,
        checkpointer=checkpointer,
        debug=debug
	)
    return agent

In [30]:
sql_agent = create_sql_agent(
    model="openai:gpt-5-nano",
    db_uri="sqlite:///data/Chinook.db",
    debug=False
)

sql_agent.invoke({
    "messages": [
        {"role": "user", "content": "Quais são as tabelas do banco de dados?"}
    ]}
)

{'messages': [HumanMessage(content='Quais são as tabelas do banco de dados?', additional_kwargs={}, response_metadata={}, id='13c94c8c-7460-4ecb-b369-959903148c40'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 469, 'prompt_tokens': 485, 'total_tokens': 954, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CmkMK29zhnvUGtpPo9KbRfZugSHEK', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b1df2-658a-77d2-b9e7-57427402d4fe-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_GVR74x88YGbnJDRcqCA8YDFb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 485, 'output_tokens': 469, 'total_token

In [33]:
sql_agent_tool = create_sql_agent(
    model="openai:gpt-5-nano",
    db_uri="sqlite:///data/Chinook.db",
    debug=False,
    as_tool=True,
)

sql_agent_tool.invoke({"question": "Quais são as tabelas do banco de dados?"})

'As tabelas do banco de dados são:\n- Album\n- Artist\n- Customer\n- Employee\n- Genre\n- Invoice\n- InvoiceLine\n- MediaType\n- Playlist\n- PlaylistTrack\n- Track\n\nSe quiser, posso exibir o esquema (colunas e tipos) de alguma tabela específica.'

In [ ]:
conversational_agent = create_conversational_agent(model="openai:gpt-5-nano", tools=[sql_agent_tool])
conversational_agent.invoke({
    "messages": [{"role": "user", "content": "Hi"}]
})

{'messages': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='a6950ce0-e861-4382-bb80-c07e3186b08e'),
  AIMessage(content='Hi there! How can I help today?\n\nI can assist with:\n- Explaining topics or answering questions\n- Writing, editing, and brainstorming\n- Coding help and debugging\n- Data analysis and creating/ interpreting queries\n- Planning, organizing, and project ideas\n\nIf you have a database and want to run a query, tell me your question and I can use SQL to answer it. What would you like to work on?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 544, 'prompt_tokens': 138, 'total_tokens': 682, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 

In [40]:
question = "Quais gêneros musicais estão disponíveis no banco de dados?"

for step in conversational_agent.stream(
    {"messages": [
		{"role": "user", "content": question},
	]},
    stream_mode="values",
):
	step["messages"][-1].pretty_print()

================================ Human Message =================================

Quais gêneros musicais estão disponíveis no banco de dados?
================================== Ai Message ==================================
Tool Calls:
  sql_agent (call_DzHFGF54eOriAnO1ToBl55In)
 Call ID: call_DzHFGF54eOriAnO1ToBl55In
  Args:
    question: Quais gêneros musicais estão disponíveis no banco de dados?
================================= Tool Message =================================
Name: sql_agent

Aqui estão os gêneros disponíveis no banco de dados (GenreId - Nome):

- 1 - Rock
- 2 - Jazz
- 3 - Metal
- 4 - Alternative & Punk
- 5 - Rock And Roll
- 6 - Blues
- 7 - Latin
- 8 - Reggae
- 9 - Pop
- 10 - Soundtrack
- 11 - Bossa Nova
- 12 - Easy Listening
- 13 - Heavy Metal
- 14 - R&B/Soul
- 15 - Electronica/Dance
- 16 - World
- 17 - Hip Hop/Rap
- 18 - Science Fiction
- 19 - TV Shows
- 20 - Sci Fi & Fantasy
- 21 - Drama
- 22 - Comedy
- 23 - Alternative
- 24 - Classical
- 25 - Opera

Deseja que eu 

In [42]:
question = """
Minha sobrinha comentou comigo sobre esse sistema outro dia porque sabe que eu sempre gostei muito de música,
mas naquele momento eu estava sem tempo e acabei não explorando direito as funcionalidades;
hoje, com mais calma, fiquei curioso para entender melhor se por acaso o Gilberto Gil aparece no sistema.
"""

for step in conversational_agent.stream(
    {"messages": [
		{"role": "user", "content": question},
	]},
    stream_mode="values",
):
	step["messages"][-1].pretty_print()

================================ Human Message =================================


Minha sobrinha comentou comigo sobre esse sistema outro dia porque sabe que eu sempre gostei muito de música,
mas naquele momento eu estava sem tempo e acabei não explorando direito as funcionalidades;
hoje, com mais calma, fiquei curioso para entender melhor se por acaso o Gilberto Gil aparece no sistema.

================================== Ai Message ==================================
Tool Calls:
  sql_agent (call_vIszxphThsOT2Ur7CpGGQm9q)
 Call ID: call_vIszxphThsOT2Ur7CpGGQm9q
  Args:
    question: Existe algum registro que contenha o nome 'Gilberto Gil' no sistema? Pode indicar se ele é artista, música, álbum ou associação.
================================= Tool Message =================================
Name: sql_agent

Sim. Existem registros com o nome Gilberto Gil no sistema.

Resultados encontrados:
- Artista
  - Nome: Gilberto Gil
  - ArtistaId: 27
- Álbum
  - Não foram encontrados títulos de ál

In [43]:
for step in conversational_agent.stream(
    {"messages": [
		{"role": "user", "content": "O artista Gilberto Gil está no sistema?"},
		{"role": "ai", "content": "Sim, o artista Gilberto Gil está no sistema."},
		{"role": "user", "content": "Quantas músicas dele aparecem?"},
	]},
    stream_mode="values",
):
	step["messages"][-1].pretty_print()

================================ Human Message =================================

Quantas músicas dele aparecem?
================================== Ai Message ==================================
Tool Calls:
  sql_agent (call_GkABMUsgJ9l2iDfI5WwabzVB)
 Call ID: call_GkABMUsgJ9l2iDfI5WwabzVB
  Args:
    question: Quantas músicas de Gilberto Gil aparecem no sistema?
================================= Tool Message =================================
Name: sql_agent

32 músicas de Gilberto Gil aparecem no sistema.

Se quiser, posso listar os títulos ou ver por álbum.
================================== Ai Message ==================================

São 32 músicas de Gilberto Gil aparecem no sistema.

Quer que eu:
- liste os títulos (em ordem alfabética ou por álbum)?
- mostre por álbum específico?
- filtre por ano de lançamento?
- indique as faixas mais populares?


In [45]:
# Exibindos as atualizações do agente
question = "Existe o Emicida no banco de dados?"

for step in conversational_agent.stream(
    {"messages": [
		# {"role": "system", "content": system_prompt},
		{"role": "user", "content": question},
	]},
    stream_mode="values",
):
	
	message = step["messages"][-1]
	if isinstance(message, AIMessage):
		tool_calls = message.tool_calls
		for tool_call in tool_calls:
			if tool_call["name"] == "sql_db_list_tables":
				print("Listing database tables")
			elif tool_call["name"] == "sql_db_schema":
				args = tool_call["args"]["table_names"]
				print(f"Checking database tables ({args})")
			elif tool_call["name"] == "sql_db_query_checker":
				print(f"Checking SQL query correctness")
			elif tool_call["name"] == "sql_db_query":
				print(f"Querying the database")
			elif tool_call["name"] == "sql_agent":
				print(f"Requisitando o SQL Agent")
			
		if not tool_calls:
			print(f"Answer:\n{message.content}")

Requisitando o SQL Agent
Answer:
Resumo direto: não, Emicida não consta na tabela Artist do banco de dados atual.

Posso fazer buscas mais amplas para ter certeza:
- Busca por correspondência exata, sem diferenciação de caixa (case-insensitive):
  - SELECT ArtistId, Name FROM Artist WHERE LOWER(Name) = 'emicida';
- Busca por parte do nome (também case-insensitive):
  - SELECT ArtistId, Name FROM Artist WHERE LOWER(Name) LIKE '%emicida%';

Também posso checar em outras tabelas relacionadas (por exemplo, Tracks associadas a artistas) se houver necessidade.

Quer que eu rode essas consultas agora? Se sim, sigo com elas e retorno os resultados.


## Adicionando histórico

In [60]:
checkpointer = InMemorySaver()

conversational_agent = create_conversational_agent(
    model="openai:gpt-5-nano",
    tools=[sql_agent_tool],
    checkpointer=checkpointer,
)

In [61]:
bool(checkpointer.get({"configurable": {"thread_id": "123"}}))

False

In [62]:
response = conversational_agent.invoke(
	input={"messages": [{"role": "user", "content": "O artista Gilberto Gil está no sistema?"}]},
	config={"configurable": {"thread_id": "123"}},
)

print(response["messages"][-1].content)

Sim. Gilberto Gil está cadastrado no sistema.

- ArtistId: 27
- Nome: Gilberto Gil

O que você quer que eu mostre agora? Algumas opções:
- Listar as faixas dele
- Mostrar os álbuns
- Exibir os gêneros associados
- Encontrar playlists que contenham faixas dele

Diga qual opção ou peça algo específico.


In [63]:
bool(checkpointer.get({"configurable": {"thread_id": "123"}}))

True

In [64]:
response = conversational_agent.invoke(
	input={"messages": [{"role": "user", "content": "Existem quantas músicas dele?"}]},
	config={"configurable": {"thread_id": "123"}},
)

print(response["messages"][-1].content)

Existem 32 músicas de Gilberto Gil cadastradas no sistema (ArtistId 27).

Quer que eu:
- liste as faixas dele,
- mostre os álbuns,
- exiba os gêneros associados,
- encontre playlists com as faixas dele?

Diga qual opção ou peça algo específico.
